In [ ]:
!pip install faiss-cpu transformers sentence-transformers gradio bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211

In [ ]:
# 挂载 Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 在 Drive 中创建存储向量数据库的文件夹
import os
db_path = '/content/drive/MyDrive/vectorstore/db_faiss'
os.makedirs(db_path, exist_ok=True)
print(f"FAISS 数据库目录：{db_path}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 113.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

In [ ]:
import pandas as pd

csv_path = '/content/Shakespeare_data.csv'
df = pd.read_csv(csv_path)

# 只保留人物台词行：PlayerLinenumber 不为空且 PlayerLine 非空
filtered_df = df[
    df['PlayerLinenumber'].notna() &
    df['PlayerLine'].astype(str).str.strip().ne('')
]
print(f"Kept {len(filtered_df)} dialogue rows out of {len(df)} total.")

# 最终用作索引的文本列表
corpus = filtered_df['PlayerLine'].astype(str).tolist()

Kept 111393 dialogue rows out of 111396 total.


In [ ]:
import faiss
from sentence_transformers import SentenceTransformer

# 初始化 embedder
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# 只对过滤后的 corpus 做嵌入
embeddings = embedder.encode(corpus, show_progress_bar=True, convert_to_numpy=True)
print("Embeddings shape:", embeddings.shape)

# 构建索引
d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings)
print("Number of vectors indexed:", index.ntotal)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/3482 [00:00<?, ?it/s]

Embeddings shape: (111393, 384)
Number of vectors indexed: 111393


In [ ]:
import os, pickle

db_path = '/content/drive/MyDrive/vectorstore/db_faiss'
os.makedirs(db_path, exist_ok=True)

faiss.write_index(index, os.path.join(db_path, 'index.faiss'))
with open(os.path.join(db_path, 'index.pkl'), 'wb') as f:
    # 同时保存过滤后的元数据，以后可用于定位答案来源
    pickle.dump({
        'texts': corpus,
        'meta': filtered_df.to_dict(orient='list')
    }, f)

print("✅ Saved filtered index and metadata.")


✅ Saved filtered index and metadata.


In [ ]:
# 安装 Hugging Face CLI（如未安装）
!pip install huggingface_hub

# 登录（会弹出链接、让你粘贴 token）
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
The token `test` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushi

In [ ]:
import os, pickle
import faiss
from sentence_transformers import SentenceTransformer

DB_DIR = '/content/drive/MyDrive/vectorstore/db_faiss'
index = faiss.read_index(os.path.join(DB_DIR, 'index.faiss'))
with open(os.path.join(DB_DIR, 'index.pkl'), 'rb') as f:
    db_data = pickle.load(f)
texts = db_data['texts']

embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


In [ ]:
# Cell 1: Preprocessing & build chunk-level FAISS index
import pandas as pd, os, pickle, numpy as np
from sentence_transformers import SentenceTransformer
import faiss

# 1) 读取并过滤对话行
df = pd.read_csv('/content/Shakespeare_data.csv')
filtered = df[df['PlayerLinenumber'].notna() & df['PlayerLine'].astype(str).str.strip().ne('')]

# 2) 按 ActSceneLine 合并为 chunk
chunks, chunk_meta = [], []
for scene, group in filtered.groupby('ActSceneLine'):
    text = "\n".join(group['PlayerLine'].astype(str).tolist())
    chunks.append(text)
    chunk_meta.append(scene)

# 3) 生成 embeddings
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = embedder.encode(chunks, show_progress_bar=True, convert_to_numpy=True)

# 4) 构建 IDMap FAISS 索引
d = embeddings.shape[1]
index = faiss.IndexIDMap(faiss.IndexFlatL2(d))
ids = np.arange(len(chunks))
index.add_with_ids(embeddings, ids)

# 5) 保存到 Google Drive
db_path = '/content/drive/MyDrive/vectorstore/db_faiss'
os.makedirs(db_path, exist_ok=True)
faiss.write_index(index, os.path.join(db_path, 'index.faiss'))
with open(os.path.join(db_path, 'index.pkl'), 'wb') as f:
    pickle.dump({'chunks': chunks, 'meta': chunk_meta}, f)

print("✅ Chunk-level FAISS index saved.")


Batches:   0%|          | 0/504 [00:00<?, ?it/s]

✅ Chunk-level FAISS index saved.


In [ ]:
# Cell 2: Mount & load index + metadata + embedder
from google.colab import drive
drive.mount('/content/drive')

import faiss, pickle, os
# 载入 chunks 和 meta
with open('/content/drive/MyDrive/vectorstore/db_faiss/index.pkl', 'rb') as f:
    data = pickle.load(f)
chunks     = data['chunks']
chunk_meta = data['meta']

# 载入 FAISS 索引
index = faiss.read_index('/content/drive/MyDrive/vectorstore/db_faiss/index.faiss')

# 复用同一个 embedder
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

print(f"Loaded {len(chunks)} chunks.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Loaded 16122 chunks.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import os

# 模型 ID 与 HF token
MODEL_ID = "MistralAI/mistral-7b-v0.1"
hf_token = os.environ.get("HUGGINGFACE_TOKEN")

# 1) 构造 8‑bit 量化配置，并启用 CPU offload
bnb_config_8bit = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True
)

# 2) 加载 tokenizer（需要 trust_remote_code）
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    trust_remote_code=True,
    use_auth_token=hf_token
)

# 3) 加载量化模型
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config_8bit,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=hf_token
)

# 4) 构建生成 pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

print("✅ 8‑bit quantized model loaded successfully.")


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0


✅ 8‑bit quantized model loaded successfully.


In [ ]:
# Load the Cross-Encoder reranker
from sentence_transformers import CrossEncoder

reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# Load other necessary components (FAISS index, sentence embedder, etc.)
from sentence_transformers import SentenceTransformer
import faiss
import pickle

# Loading the FAISS index and metadata (Make sure this points to your actual file paths)
with open('/content/drive/MyDrive/vectorstore/db_faiss/index.pkl', 'rb') as f:
    data = pickle.load(f)
chunks, chunk_meta = data['chunks'], data['meta']

embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Load the FAISS index
index = faiss.read_index('/content/drive/MyDrive/vectorstore/db_faiss/index.faiss')

# Now you can proceed with the rag_query function

def rag_query(question: str,
              act_scene_prefix: str,
              top_k: int = 20,
              rerank_k: int = 5) -> str:
    valid_ids = [i for i, scene in enumerate(chunk_meta)
                 if scene.startswith(act_scene_prefix)]
    if not valid_ids:
        return f"No context found for scene `{act_scene_prefix}`."

    q_emb = embedder.encode([question], convert_to_numpy=True)
    D, I = index.search(q_emb, top_k)
    candidates = [(idx, D[0][j])
                  for j, idx in enumerate(I[0])
                  if idx in valid_ids]

    if candidates:
        candidates = sorted(candidates, key=lambda x: x[1])[:top_k]
        cand_ids    = [idx for idx, _ in candidates]
        cand_chunks = [chunks[i] for i in cand_ids]
    else:
        cand_ids    = valid_ids[:rerank_k]
        cand_chunks = [chunks[i] for i in cand_ids]

    # Cross-Encoder for reranking
    if len(cand_chunks) > 1:
        pairs  = [(question, c) for c in cand_chunks]
        scores = reranker.predict(pairs)
        ranked = sorted(
            zip(cand_ids, cand_chunks, scores),
            key=lambda x: x[2], reverse=True
        )[:rerank_k]
        context = "\n".join(text for _, text, _ in ranked)
    else:
        context = cand_chunks[0] if cand_chunks else ""

    # Constructing the prompt and generating the answer
    few_shot = (
        "You are a Shakespeare scholar. Provide concise, evidence‑based answers.\n"
        "Example:\n"
        "Q: What do the witches predict in Act I, Scene III?\n"
        "A: They predict Macbeth will become king and Banquo's descendants will inherit the throne.\n"
        "-----\n"
    )
    prompt = (
        f"{few_shot}"
        f"Context:\n{context}\n\n"
        f"Question: {question}\n"
        "Answer:"
    )

    # Assuming generator is already initialized and available
    out = generator(
        prompt,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.2,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        return_full_text=False
    )
    return out[0]['generated_text']

# Test the updated function
print(rag_query("Who speaks first in Act I?", act_scene_prefix="1.1"))


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

 In Act I, the First Quarto has Cassius speaking the first lines (which are spoken by Brutus in the Folio), while the Folio has Lucius speaking the opening lines of the scene.


In [ ]:
import gradio as gr
import numpy as np # Ensure numpy is imported if not already

# --- RAG related functions and loaded variables from previous cells ---
# Ensure the necessary functions and variables (index, chunks, chunk_meta,
# embedder, generator, tokenizer, rag_query) are defined and loaded
# correctly from the previous cells before running this block.

# Example placeholder for rag_query for testing layout:
# def rag_query(question: str, act_scene_prefix: str, top_k: int = 20, rerank_k: int = 5) -> str:
#     # print(f"Simulating RAG query for: '{question}' in scene {act_scene_prefix}")
#     # Simulate context retrieval and generation
#     # context = f"Retrieved context for '{question}' in scene {act_scene_prefix}..."
#     # Simulate LLM generation
#     return f"This is a simulated answer to: {question}"
# --- End of placeholder/copied code ---


# Gradio callback function to handle user input and return updated chat history
def respond(message: str, chat_history: list) -> tuple[str, list]:
    """
    Handles user input, queries the RAG model, and updates chat history.

    Args:
        message: The user's input message.
        chat_history: The current list of conversation turns in 'messages' format.
                      Example: [{'role': 'user', 'content': 'Hi'}, {'role': 'assistant', 'content': 'Hello!'}]

    Returns:
        A tuple containing an empty string (to clear the input box)
        and the updated chat_history list.
    """
    # --- RAG Query ---
    # Note: act_scene_prefix is hardcoded here, might need dynamic input later
    act_scene_prefix="1.1" # Or make this dynamic if needed
    bot_message = rag_query(message, act_scene_prefix=act_scene_prefix)
    # --- End RAG Query ---

    # Append the user message and the bot reply to the history in the 'messages' format
    chat_history.append({"role": "user", "content": message})
    chat_history.append({"role": "assistant", "content": bot_message})

    # Return an empty string to clear the input box, and the updated history
    return "", chat_history

# Callback function to clear the chat history
def clear_chat() -> tuple[None, list]:
    """Clears the chatbot history and the input message box."""
    return None, [] # Return None for the message box, empty list for chatbot

# ----- Gradio Interface Build -----
with gr.Blocks(title="Shakespeare RAG QA", theme=gr.themes.Default()) as demo:
    gr.Markdown("## Ask questions about Shakespeare's works (Multi-turn chat)")

    # Chat display area using the recommended 'messages' type
    chatbot = gr.Chatbot(
        label="Conversation",
        height=500,
        type="messages" # Use the recommended message format
    )

    # Input and button area, arranged horizontally using Row
    with gr.Row():
        # Text input box
        msg = gr.Textbox(
            label=None, # Hide the label
            placeholder="Type your message here...",
            show_label=False,
            lines=1, # Single-line input
            scale=7, # Input box takes more horizontal space
            container=False # Remove default container border for a cleaner look
        )
        # Send button, primary style might make it colored (orange in your screenshot, blue in default Gradio)
        send_btn = gr.Button("Send", variant="primary", scale=1)
        # Clear button, English label, default style (usually light gray)
        clear_btn = gr.Button("Clear", scale=1)

    # ----- Event Handling -----

    # Handle Send button click / Enter key submission in the textbox
    # 1. Trigger on Send button click
    send_btn.click(
        fn=respond,
        inputs=[msg, chatbot],       # Inputs: user message textbox, chat history
        outputs=[msg, chatbot]       # Outputs: clear message textbox, update chat history
    )
    # 2. Trigger on Enter key press in the textbox (same action as Send)
    msg.submit(
        fn=respond,
        inputs=[msg, chatbot],
        outputs=[msg, chatbot]
    )

    # Handle Clear button click
    clear_btn.click(
        fn=clear_chat,               # Call the clear function
        inputs=[],                   # No inputs needed
        outputs=[msg, chatbot]       # Clear both the message textbox and the chat history
    )

# Launch the Gradio App
# Added debug=True to show detailed errors in Colab output if any occur
demo.launch(share=True, debug=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7957d0f571babbad1b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
